In [1]:
from choice_model import ChoiceModel

folder = 'orig_graph'

def predict(period):
    model = ChoiceModel()
    prev_period_file = f'{folder}/{folder}_{period-1}.pkl'
    new_model = ChoiceModel(graph_path=prev_period_file) 
    graph = model.get_period(period)
    new_nodes = [n for n in graph.nodes if n not in list(new_model.graph.nodes)]
    # new_nodes = [n for n in new_nodes if graph.nodes[n]['type'] != 'Long-term Project']
    test_graph = new_model.graph.copy()
    test_graph = model.predict_links(test_graph, new_nodes, file_name=f"{folder}/{folder}_{period}.pkl",period=period)
    new_model = ChoiceModel(graph_path=f"{folder}/{folder}_{period}.pkl")
    issolated_nodes = new_model.get_isolated_nodes()
    if len(issolated_nodes) > 0:
        test_graph = new_model.graph.copy()
        test_graph = model.predict_links(test_graph, issolated_nodes,file_name=f"{folder}/{folder}_{period}.pkl",k1=10,k2=5,top_k=None)
        new_model.remove_isolated_nodes()
    new_model.visualize(file_name=f"{folder}/{folder}_{period}.html")
    PPI, CVI, DCI = new_model.evaluate(test_graph)
    print(f"Period {period} PPI: {PPI} CVI: {CVI} DCI: {DCI}")


In [2]:
for i in range(1,5):
    predict(i)

100%|██████████| 76/76 [00:00<00:00, 659.64it/s]


Error raised by inference endpoint: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/embeddings (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x17c447340>: Failed to establish a new connection: [Errno 61] Connection refused'))
Error raised by inference endpoint: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/embeddings (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x17c447be0>: Failed to establish a new connection: [Errno 61] Connection refused'))
Error raised by inference endpoint: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/embeddings (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x17c447610>: Failed to establish a new connection: [Errno 61] Connection refused'))
Error raised by inference endpoint: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /

  0%|          | 0/136 [00:00<?, ?it/s]

Error raised by inference endpoint: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/embeddings (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x17c4d1fa0>: Failed to establish a new connection: [Errno 61] Connection refused'))


100%|██████████| 136/136 [00:00<00:00, 1116.85it/s]


...

Error raised by inference endpoint: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/embeddings (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1740cbc40>: Failed to establish a new connection: [Errno 61] Connection refused'))
Error raised by inference endpoint: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/embeddings (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x17c4cfb50>: Failed to establish a new connection: [Errno 61] Connection refused'))
Error raised by inference endpoint: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/embeddings (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x17c4cf940>: Failed to establish a new connection: [Errno 61] Connection refused'))
Period 3 PPI: 0.31012110726643594 CVI: 2.843137254901961 DCI: 0.058823529411764705


  0%|          | 0/123 [00:00<?, ?it/s]

Error raised by inference endpoint: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/embeddings (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x17f89e880>: Failed to establish a new connection: [Errno 61] Connection refused'))
Error raised by inference endpoint: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/embeddings (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x17f89ea90>: Failed to establish a new connection: [Errno 61] Connection refused'))
Error raised by inference endpoint: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/embeddings (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x17f5c21c0>: Failed to establish a new connection: [Errno 61] Connection refused'))
Error raised by inference endpoint: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /

100%|██████████| 123/123 [00:00<00:00, 1110.94it/s]

Error raised by inference endpoint: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/embeddings (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x17c4bf970>: Failed to establish a new connection: [Errno 61] Connection refused'))
Error raised by inference endpoint: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/embeddings (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x17f5c2550>: Failed to establish a new connection: [Errno 61] Connection refused'))
Error raised by inference endpoint: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/embeddings (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x17c4d6ac0>: Failed to establish a new connection: [Errno 61] Connection refused'))
Error raised by inference endpoint: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /